# Funciones de conexion con CHEMBL


Escribo una funcion para poder extraer datos de chembl-mysql directo desde python

In [1]:
import pandas as pd
import numpy as np

def mysql_pull(ip:str,ssh_usr:str,mysql_usr:str,ssh_pswd:str,mysql_pswd:str,db,query,mysql_ip:str='127.0.0.1',mysql_port:int = 3306):
    from sshtunnel import SSHTunnelForwarder
    from sqlalchemy import create_engine

    server = SSHTunnelForwarder(
        (ip, 22),
        ssh_username=ssh_usr,
        ssh_password=ssh_pswd,
        remote_bind_address=(mysql_ip, mysql_port)
        )

    server.start()
    local_port = str(server.local_bind_port)
    engine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(mysql_usr, mysql_pswd, mysql_ip, local_port, db))

    dataDF = pd.read_sql(query, con=engine)

    server.stop()
    return dataDF

# Funciones de limpieza y parseo

En funcion de los criterios de la publicacion y propios de los datos, defino una funcion que hace toda la limpieza, El proceso esta está mucho más detallado en data_procesing.ipynb

In [9]:
def deepscreen_cleaning(df_raw):
    df = df_raw.copy()

    col_names = {
    'compund_chembl_id':'comp_id',
    'target_chemb_id':'target',
    'standard_relation':'relation',
    'standard_units':'units',
    'standard_type':'type',
    'potential_duplicate':'potential_duplicates',
    'target_sequence':'sequence'
    }

    df = df.rename(columns=col_names)

    print(f'Entries previous null biactivity deletion {len(df.index)}')
    df = df[~df.bioactivity.isnull()]
    print(f'Entries after null biactivity deletion {len(df.index)}')

    print(f'Entries previous bioactivity unification {len(df.index)}')
    df = df.groupby(['target','comp_id','relation','type','smiles','sequence','molregno','units'],as_index=False)['bioactivity'].median()
    print(f'Entries previous bioactivity unification {len(df.index)}')

    print(f'Entries previous bioactivity binarization {len(df.index)}')
    def step_func_bioactivity(x):
        if x <= 10000: 
            return 1 
        elif x >= 20000:
            return 0
        else:
            return pd.NA
    df['binary_bioactivity'] = df['bioactivity'].apply(step_func_bioactivity)
    df = df[~df.binary_bioactivity.isnull()]
    print(f'Entries previous bioactivity binarization {len(df.index)}')

    print(f'Entries previous assay type conflict cleaning {len(df.index)}')
    # Se conserva solo los que tengan una bioactividad binaria igual para todos los ensayos
    df = df.groupby(['comp_id','target','relation'],as_index=False).filter(lambda g: (g['binary_bioactivity'].nunique() == 1))
    # Se descartan los duplicados y se conserva aleatoriamente el primer resultado
    df = df.drop_duplicates(subset=['comp_id','target','relation'],keep='first')
    print(f'Entries after assay type conflict cleaning {len(df.index)}')

    print(f'Entries previous relation conflict cleaning {len(df.index)}')
    df = df.drop_duplicates(subset=['comp_id','target'],keep=False)
    print(f'Entries previous relation type conflict cleaning {len(df.index)}')

    return df

In [3]:
def pivot_to_trypanosomatcs_deepscreen(df):
    df_pivot = df.copy()
    df_pivot = df_pivot[['target','comp_id','binary_bioactivity','smiles']].pivot(index='comp_id',columns='target',values='binary_bioactivity')
    df_pivot = df_pivot.reset_index()
    df_pivot = pd.merge(df_pivot,df[['comp_id','smiles']],how='left',on='comp_id')
    df_pivot = df_pivot.drop_duplicates()
    return df_pivot

# Funciones de extraccion de datos de autores

In [4]:
import sqlalchemy as sa

def author_json_extract(json_path:str,target_id:str,ssh_pass:str,mysql_pass:str,db:str):
    df = pd.read_json(json_path, orient='index')
    df = df.transpose()
    df = pd.concat([df.training,df.test,df.validation])
    df = df.dropna()
    df = pd.DataFrame(df.to_list())
    df = df.rename(columns={0:'comp_id',1:target_id})
    comp_id_tuple = tuple(df.comp_id.to_list())
    df_smiles = get_smiles_chembl(comp_id_tuple, ssh_pass=ssh_pass,mysql_pass=mysql_pass,db=db)
    df = pd.merge(df,df_smiles,how='left',left_on='comp_id',right_on='comp_id')
    return df

def get_smiles_chembl(comp_id_tuple:tuple, ssh_pass:str, mysql_pass:str, db:str = 'chembl_31'):
    '''given a tuple with comp_id it returns a pandas dataframe with all the compounds chembl id and its canonical smiles'''
    query_smiles = f'''
SELECT md.chembl_id, cmpstc.canonical_smiles
FROM compound_structures AS cmpstc 
JOIN molecule_dictionary AS md ON md.molregno = cmpstc.molregno
WHERE md.chembl_id IN {tuple(comp_id_tuple)}'''
    df_smiles = mysql_pull('rho','sjinich','sjinich',ssh_pass,mysql_pass,db=db,query=query_smiles)
    df_smiles = df_smiles.rename(columns={'chembl_id':'comp_id','canonical_smiles':'smiles'})
    return df_smiles

# Obtencion de datos crudos de chembl
Defino el query que se usa para extraer la informacion de chembl

In [ ]:
query = '''SELECT act.molregno, md.chembl_id AS compund_chembl_id,
        trgd.chembl_id AS target_chemb_id,
        act.standard_relation,
        act.standard_value AS bioactivity,
        act.standard_units,
        act.standard_type,
        act.potential_duplicate,
        cmpstc.canonical_smiles AS smiles,
        cs.sequence AS target_sequence
FROM activities AS act
JOIN (SELECT tid, assay_id, assay_type, chembl_id FROM assays) AS ass ON act.assay_id = ass.assay_id
JOIN target_dictionary AS trgd ON ass.tid = trgd.tid
JOIN compound_structures AS cmpstc ON act.molregno = cmpstc.molregno
JOIN molecule_dictionary AS md ON act.molregno = md.molregno
JOIN target_components AS tc ON ass.tid = tc.tid
JOIN component_sequences AS cs ON tc.component_id = cs.component_id
WHERE (trgd.target_type = 'SINGLE PROTEIN')
    AND (ass.assay_type ='B')
    AND (act.standard_type IN ('IC50','EC50','AC50','Ki','Kd','Potency'))
    AND (act.pchembl_value IS NOT NULL)
;'''

In [ ]:
from getpass import getpass

pass_mysql = getpass('mysql pass:')
pass_ssh = getpass('ssh pass:')
df_chembl_23_filter_raw = mysql_pull('rho','sjinich','sjinich',pass_ssh,pass_mysql,'chembl_23',query)
df_chembl_23_filter_raw

In [ ]:
df_chembl_23_filter_raw.to_pickle('df_chembl_23.pickle')

In [ ]:

pass_mysql = getpass('mysql pass:')
pass_ssh = getpass('ssh pass:')
df_chembl_31_filter_raw = mysql_pull('rho','sjinich','sjinich',pass_ssh,pass_mysql,'chembl_31',query)
df_chembl_31_filter_raw

In [ ]:
df_chembl_31_filter_raw.to_pickle('df_chembl_31.pickle')

In [ ]:
authors_zip_file = '/home/sebastian-wsl/DEEPScreen/data_procesing_filter/data_autores/target_training_datasets.zip'

import zipfile
from io import BytesIO
from os import path

import getpass
pass_ssh = getpass.getpass('ssh pass:')
pass_mysql = getpass.getpass('mysql_pass:')

folder = '/home/sebastian-wsl/DEEPScreen/data_procesing_filter/data_autores/'

with zipfile.ZipFile(authors_zip_file,mode='r') as archive:
    files = archive.namelist()
    del files[0]
    for file in files:
        comp_id = file[:file.find('.zip')]
        filedata = BytesIO(archive.read(file))
        with zipfile.ZipFile(filedata) as archive_in:
            filedata_json = BytesIO(archive_in.read(path.join(comp_id,'train_val_test_dict.json')))
            df = author_json_extract(filedata_json,comp_id,pass_ssh,pass_mysql,'chembl_23')
            df.to_pickle(path.join(folder,comp_id+'.pickle'))
            

In [ ]:
df_test = pd.read_pickle('/home/sebastian-wsl/DEEPScreen/data_procesing_filter/data_autores/CHEMBL1075104.pickle')

In [ ]:
df_test[df_test.comp_id == 'CHEMBL88962']['smiles'].values

# Comparaciones de datasets

In [5]:
df_chembl_23_raw = pd.read_pickle('./df_chembl_23.pickle')

In [10]:
df_chembl_23_filter = deepscreen_cleaning(df_chembl_23_raw)
df_chembl_23_filter

Entries previous null biactivity deletion 993748
Entries after null biactivity deletion 993748
Entries previous bioactivity unification 993748
Entries previous bioactivity unification 845635
Entries previous bioactivity binarization 845635
Entries previous bioactivity binarization 789109
Entries previous assay type conflict cleaning 789109
Entries after assay type conflict cleaning 762742
Entries previous relation conflict cleaning 762742
Entries previous relation type conflict cleaning 762742


,target,comp_id,relation,type,smiles,sequence,molregno,units,bioactivity,binary_bioactivity
0,CHEMBL1075021,CHEMBL36506,=,IC50,CO[C@@H]1[C@@H](OC(=O)N)[C@@H](O)[C@H](Oc2ccc3...,MTEEIKNLQAQDYDASQIQVLEGLEAVRMRPGMYIGSTSKEGLHHL...,48697,nM,37.0,1
1,CHEMBL1075021,CHEMBL565572,=,IC50,CCNC(=O)Nc1cn2c(cc(cc2n1)c3cccnc3)c4ncc(cn4)C(...,MTEEIKNLQAQDYDASQIQVLEGLEAVRMRPGMYIGSTSKEGLHHL...,576354,nM,163.0,1
2,CHEMBL1075021,CHEMBL565741,=,IC50,CCC(=O)Nc1cn2c(cc(cc2n1)c3cccnc3)c4ncc(C)cn4,MTEEIKNLQAQDYDASQIQVLEGLEAVRMRPGMYIGSTSKEGLHHL...,576405,nM,255.0,1
3,CHEMBL1075021,CHEMBL565743,=,IC50,CCNC(=O)Nc1cn2c(cc(cc2n1)C3=CC(=O)N(CCF)C=C3)c...,MTEEIKNLQAQDYDASQIQVLEGLEAVRMRPGMYIGSTSKEGLHHL...,576468,nM,376.0,1
4,CHEMBL1075021,CHEMBL565750,=,IC50,CCNC(=O)Nc1cn2c(cc(cc2n1)C3=CC(=O)N(CC)C=C3)c4...,MTEEIKNLQAQDYDASQIQVLEGLEAVRMRPGMYIGSTSKEGLHHL...,576407,nM,51.0,1
...,...,...,...,...,...,...,...,...,...,...
845630,CHEMBL6198,CHEMBL584189,=,Ki,Cc1cccc(C)c1OCC(=O)NC[C@](O)(Cc2ccccc2)C(=O)N3...,MDIAVKEQDYSNGLIKNSAAFENLKFSNIKNFKVQKRFQILYYILF...,570359,nM,190.0,1
845631,CHEMBL6198,CHEMBL584190,=,Ki,Cc1cccc(C)c1OCC(=O)NC[C@](O)(CCc2ccccc2)C(=O)N...,MDIAVKEQDYSNGLIKNSAAFENLKFSNIKNFKVQKRFQILYYILF...,570361,nM,340.0,1
845632,CHEMBL6198,CHEMBL584191,=,Ki,Cc1cccc(C)c1OCC(=O)NC[C@@](O)(Cc2ccccc2)C(=O)N...,MDIAVKEQDYSNGLIKNSAAFENLKFSNIKNFKVQKRFQILYYILF...,570362,nM,160.0,1
845633,CHEMBL6198,CHEMBL584192,=,Ki,Cc1cccc(C)c1OCC(=O)NC[C@](O)(Cc2ccccc2)C(=O)N3...,MDIAVKEQDYSNGLIKNSAAFENLKFSNIKNFKVQKRFQILYYILF...,570363,nM,120.0,1


In [14]:
authors_chembl286 = pd.read_pickle('/home/sebastian-wsl/DEEPScreen/data_procesing_filter/data_autores/CHEMBL286.pickle')
authors_chembl286

,comp_id,CHEMBL286,smiles
0,CHEMBL1644461,1,CC(C)[C@H](C[C@H](O)[C@H](COCc1ccccc1)NC(=O)c2...
1,CHEMBL339114,1,CC(C)(C)OC(=O)NC(Cc1ccccc1)C(=O)N[C@H]2CCC(=O)...
2,CHEMBL3401538,1,COCCCOc1cc(ccc1OC)C(=O)N(C[C@@H]2CNC[C@H]2NS(=...
3,CHEMBL1825183,1,Cc1c(F)cccc1Cc2c(C(=O)N3CCNCC3)c4ccncc4n2c5ccccc5
4,CHEMBL584509,0,NC1=N[C@@](C2=NCCCN12)(c3ccc(OC(F)(F)F)cc3)c4c...
...,...,...,...
1847,CHEMBL1165196,0,COc1ccc(cc1Cl)N2N=C(C(=O)NCC(=O)Nc3cccc(C)n3)c...
1848,CHEMBL1454208,0,Fc1ccc(cc1)C(=O)Nc2ccc(Cl)nc2
1849,CHEMBL3354251,0,CN1C(=N)N[C@](CCC2CCCCC2)(C[C@H]3CCC[C@H](C3)N...
1850,CHEMBL1256491,1,Cc1cc(Cl)c(OCCOc2ccc(CC(CN)c3ccc(cc3C)c4ccccc4...


In [13]:
chembl286 = df_chembl_23_filter[df_chembl_23_filter.target == 'CHEMBL286']
chembl286_pivot = pivot_to_trypanosomatcs_deepscreen(chembl286)
chembl286_pivot

,comp_id,CHEMBL286,smiles
0,CHEMBL100605,1,CCCC[C@H](N[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
1,CHEMBL100647,1,CCCC[C@H](O[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
2,CHEMBL100800,1,CCCC[C@H](N[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
3,CHEMBL100924,1,CC(C)CC(NC(=O)[C@H](Cc1c[nH]cn1)NC(=O)[C@H](Cc...
4,CHEMBL100946,1,CCCC[C@H](O[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
...,...,...,...
2630,CHEMBL99651,1,CCCC[C@H](O[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
2631,CHEMBL99813,1,CCCC[C@H](N[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
2632,CHEMBL99858,1,CCCC[C@H](O[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
2633,CHEMBL99903,1,CCCC[C@H](O[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...


In [18]:
chembl286_pos_mio = chembl286_pivot[chembl286_pivot.CHEMBL286 == 1]

In [19]:
chembl286_pos_auth = authors_chembl286[authors_chembl286.CHEMBL286==1]

In [26]:
postive_comparison = pd.merge(chembl286_pos_auth,chembl286_pos_mio,how='outer', on='comp_id')

In [31]:
postive_comparison[postive_comparison.comp_id == 'CHEMBL99989']

,comp_id,CHEMBL286_x,smiles_x,CHEMBL286_y,smiles_y
2665,CHEMBL99989,NaN,NaN,1,CCCC[C@H](N[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...


In [41]:
chembl286_pos_auth.sort_values('comp_id').head(40)

,comp_id,CHEMBL286,smiles
1203,CHEMBL101074,1,CCCC[C@H](O[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
1655,CHEMBL103463,1,CCCC[C@H](N[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
339,CHEMBL103734,1,CCCC[C@H](O[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
1223,CHEMBL103891,1,CCCC[C@H](O[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
1571,CHEMBL103929,1,CCCC[C@H](O[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
763,CHEMBL103938,1,CCCC[C@H](N[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
1111,CHEMBL1076443,1,Fc1ccc(F)c(OCCCc2ccc(cc2)C3=C([C@H]4COC[C@H](C...
1734,CHEMBL1076445,1,Fc1ccc(F)c(OCCCc2ccc(cc2)C3=C([C@H]4CS(=O)(=O)...
486,CHEMBL1076448,1,COc1cccc(CN(C2CC2)C(=O)C3=C(c4ccc(CCCOc5c(F)cc...
1286,CHEMBL1076453,1,COc1cccc(CN(C2CC2)C(=O)C3=C(C[C@H]4CNC[C@H]3N4...


In [40]:
chembl286_pos_mio.sort_values('comp_id').head(40)

,comp_id,CHEMBL286,smiles
0,CHEMBL100605,1,CCCC[C@H](N[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
1,CHEMBL100647,1,CCCC[C@H](O[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
2,CHEMBL100800,1,CCCC[C@H](N[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
3,CHEMBL100924,1,CC(C)CC(NC(=O)[C@H](Cc1c[nH]cn1)NC(=O)[C@H](Cc...
4,CHEMBL100946,1,CCCC[C@H](O[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
5,CHEMBL100949,1,CCCC[C@H](O[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
6,CHEMBL100965,1,CC(C)CC(NC(=O)[C@H](Cc1c[nH]cn1)NC(=O)[C@@H](N...
7,CHEMBL101074,1,CCCC[C@H](O[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
8,CHEMBL101075,1,CCCC[C@H](N[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
9,CHEMBL101181,1,CCCC[C@H](N[C@@H](Cc1ccccc1)C(=O)N2CCC(CC2)OCO...
